### 원본 테스트 데이터셋을 4개로 나눠 각각 query-make하기

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

In [10]:
from datasets import load_from_disk

ds = load_from_disk("../dataset/")['test']
ds1 = ds.select(range(500))
ds2 = ds.select(range(500, 1000))
ds3 = ds.select(range(1000, 1500))
ds4 = ds.select(range(1500, 2137))

ds1

Dataset({
    features: ['id', 'casetype', 'casename', 'statutes', 'facts'],
    num_rows: 500
})

In [ ]:
from query_maker import GPTQueryMaker


prompt = """당신은 복잡한 범죄 사실들을 마치 일반인이 얘기하듯 자연스러운 문장으로 풀어쓰는 한국어 챗봇입니다.
제가 범죄사실을 말하면, 당신은 실제 변호사에게 물어보듯 해당 범죄사실을 자연스러운 문장으로 풀어쓰세요. 
이때 다음과 같은 규칙들을 반드시 준수해야합니다.
1. 당신이 범죄 사실을 풀어쓸 때 담당할 역할을 피의자, 피해자, 혹은 그 주변인물 중 무작위로 정하세요.
2. 다음의 범죄 사실에을 법률 전문가가 아닌 일반인이 물어보듯 자연스럽게 다시 풀어쓰세요.
3. 이때, 500자 이내로 서술하세요. 그리고 매번 범죄사실이 주어질 때 마다 다른 사람이 질문하듯 감정, 역할, 말투, 법률 지식 수준 등을 바꿔주세요.
4. 마지막에는 실제로 당신이 담당한 역할을 맡은 사람(피의자, 피해자, 혹은 그 주변인물)이 변호사에게 해당 범죄의 경우 어떤 법령에 의해서 처벌을 받는지 법률상담을 요청하는 다양한 질문들을 덧붙여주세요. 
5. 1인칭으로 대답하세요.

범죄 사실은 다음과 같습니다:
{crime_facts}"""


query_maker = GPTQueryMaker(prompt=prompt)

for ds_part in [ds1, ds2, ds3, ds4]:
    ds_part_with_query = query_maker.generate_querys_from_datasets(ds_part, new_col_name='query', from_col_name="facts")
    ds_part_with_query.save_to_disk(f'./{ds_part}/')

### query-make된 4개의 데이터셋 조각들(`ds1~4`)를 다시 하나로 합치기

In [3]:
from datasets import load_from_disk, concatenate_datasets

ds1 = load_from_disk("./ds1/")
ds2 = load_from_disk("./ds2/")
ds3 = load_from_disk("./ds3/")
ds4 = load_from_disk("./ds4/")

ds1, ds2, ds3, ds4

(Dataset({
     features: ['id', 'casetype', 'casename', 'statutes', 'facts', 'query'],
     num_rows: 500
 }),
 Dataset({
     features: ['id', 'casetype', 'casename', 'statutes', 'facts', 'query'],
     num_rows: 500
 }),
 Dataset({
     features: ['id', 'casetype', 'casename', 'statutes', 'facts', 'query'],
     num_rows: 500
 }),
 Dataset({
     features: ['id', 'casetype', 'casename', 'statutes', 'facts', 'query'],
     num_rows: 637
 }))

In [5]:
ds = concatenate_datasets([ds1, ds2, ds3, ds4])
ds

Dataset({
    features: ['id', 'casetype', 'casename', 'statutes', 'facts', 'query'],
    num_rows: 2137
})

### 원본 데이터셋(test, val, train)에 test_with_query 데이터셋도 추가하고, 다시 저장하기

In [8]:
whole_ds = load_from_disk("../dataset/")
whole_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'casetype', 'casename', 'statutes', 'facts'],
        num_rows: 13317
    })
    validation: Dataset({
        features: ['id', 'casetype', 'casename', 'statutes', 'facts'],
        num_rows: 2376
    })
    test: Dataset({
        features: ['id', 'casetype', 'casename', 'statutes', 'facts'],
        num_rows: 2137
    })
})

In [9]:
whole_ds['test_with_query'] = ds
whole_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'casetype', 'casename', 'statutes', 'facts'],
        num_rows: 13317
    })
    validation: Dataset({
        features: ['id', 'casetype', 'casename', 'statutes', 'facts'],
        num_rows: 2376
    })
    test: Dataset({
        features: ['id', 'casetype', 'casename', 'statutes', 'facts'],
        num_rows: 2137
    })
    test_with_query: Dataset({
        features: ['id', 'casetype', 'casename', 'statutes', 'facts', 'query'],
        num_rows: 2137
    })
})

In [11]:
whole_ds.save_to_disk("../dataset/")

Saving the dataset (1/1 shards): 100%|██████████| 2137/2137 [00:00<00:00, 175330.14 examples/s]


In [16]:
ds = load_from_disk("../dataset/")
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'casetype', 'casename', 'statutes', 'facts'],
        num_rows: 13317
    })
    validation: Dataset({
        features: ['id', 'casetype', 'casename', 'statutes', 'facts'],
        num_rows: 2376
    })
    test: Dataset({
        features: ['id', 'casetype', 'casename', 'statutes', 'facts'],
        num_rows: 2137
    })
    test_with_query: Dataset({
        features: ['id', 'casetype', 'casename', 'statutes', 'facts', 'query'],
        num_rows: 2137
    })
})